In [1]:
from IPython.display import display, Math

print("We find the ground state energy lower bound (LB) with an SDP relaxation \n")

print("example model: transverse field Ising model in 1D (periodic boundary conditions) \n")

We find the ground state energy lower bound (LB) with an SDP relaxation 

example model: transverse field Ising model in 1D (periodic boundary conditions) 



In [2]:
display(Math(r' H =  J \sum_{i=1}^{L} \sigma_i^x \sigma_{i+1}^x + h \sum_{i=1}^{L} \sigma_i^z '))


<IPython.core.display.Math object>

In [3]:
print("For illustrative purposes, we first consider just two qubits, i.e. L=2")

For illustrative purposes, we first consider just two qubits, i.e. L=2


In [4]:


relax="M3"  #choose a relaxation 
symm="no"   #impose translational invariance?
print("we are choosing relaxation with ", relax)
print("are we imposing translational invariance? ", symm)



h=1; J=1; #define hamiltonian parameters


import cvxpy as cp
import numpy as np

#define sdp viariables

#first row and column of the moment matrix M
z1 = cp.Variable((1, 1), complex=True)
z2 = cp.Variable((1, 1), hermitian=True)
x1x2 = cp.Variable((1, 1), hermitian=True)
#
#they appear in the middle of M because of Pauli replacement rules
z1z2 = cp.Variable((1, 1), hermitian=True)
y1x2 = cp.Variable((1, 1), hermitian=True)
x1y2 = cp.Variable((1, 1), hermitian=True)   

#Moment matrix M
M=cp.bmat([[1 , z1[0][0], z2[0][0], x1x2[0][0]],
        [z1[0][0] , 1 ,z1z2[0][0] , 1j*y1x2[0][0]] ,
        [z2[0][0] , z1z2[0][0] , 1 , 1j*x1y2[0][0]] ,
        [x1x2[0][0] ,-1j*y1x2[0][0] ,-1j*x1y2[0][0] , 1 ]])

#M1, cancelling last row/column
M1=cp.bmat([[1 ,z1[0][0],z2[0][0]],
        [z1[0][0], 1 ,z1z2[0][0]],
        [z2[0][0], z1z2[0][0], 1 ]])

#M2, cancelling penultimate row/column
M2=cp.bmat([[1 , z1[0][0], x1x2[0][0]],
        [z1[0][0] ,1 , 1j*y1x2[0][0]],
        [x1x2[0][0] ,-1j*y1x2[0][0],1]])

#M3, cancelling the second row/column
M3=cp.bmat([[1,z2[0][0], x1x2[0][0]],
        [z2[0][0], 1, 1j*x1y2[0][0]],
        [x1x2[0][0],-1j*x1y2[0][0],1]])
 
#define constraints for the moments
constraints = []

if symm=="yes": constraints += [z1==z2]  # iff it is "yes" enforce translational invarinance 


if relax=="M":
    constraints += [M >> 0]
    constraints += [M==M.T.conj()]  #just a check, this is useless

if relax=="M1":
    constraints += [M1 >> 0]
    constraints += [M1==M1.T.conj()]  #just a check, this is useless

if relax=="M2":
    constraints += [M2 >> 0]
    constraints += [M2==M2.T.conj()]  #just a check, this is useless

if relax=="M3":
    constraints += [M3 >> 0]
    constraints += [M3==M3.T.conj()]  #just a check, this is useless

if relax=="M2,M3":
    constraints += [M2 >> 0]
    constraints += [M2==M2.T.conj()]  #just a check, this is useless
    constraints += [M3 >> 0]
    constraints += [M3==M3.T.conj()]  #just a check, this is useless


#let us bound the values of the moments btween -1 and 1
constraints += [cp.abs(z1) <= 1]
constraints += [cp.abs(z2) <= 1]
constraints += [cp.abs(x1x2) <= 1]
#
constraints += [cp.abs(x1y2) <= 1]
constraints += [cp.abs(z1z2) <= 1]
constraints += [cp.abs(y1x2) <= 1]


#define the problem
prob = cp.Problem(cp.Minimize(cp.real(h*(z1+z2) + J*x1x2)), constraints)
prob.solve(solver=cp.MOSEK, verbose=False)#set verbose to true to see details on the calculation

print("problem status is, ",prob.status)
print("The gnd state energy LB computed with the sdp relaxation is", prob.value)





we are choosing relaxation with  M3
are we imposing translational invariance?  no
problem status is,  optimal
The gnd state energy LB computed with the sdp relaxation is -2.414213562373095


In [5]:
#compare with the true value

import numpy as np


# Define basic single-qubit Pauli matrices, same as above
I_mat = np.array([[1, 0], [0, 1]])
X_mat = np.array([[0, 1], [1, 0]])
Y_mat = np.array([[0, -1j], [1j, 0]])
Z_mat = np.array([[1, 0], [0, -1]])
 

# Full Hamiltonian: H = h*(Z1 + Z2) + J*X1X2
H = h * (np.kron(Z_mat, I_mat) + np.kron(I_mat, Z_mat)) + J * np.kron(X_mat, X_mat)

# Diagonalize the Hamiltonian
eigvals, eigvecs = np.linalg.eigh(H)

# Ground state energy and eigenvector
ground_energy = eigvals[0]
#ground_state = eigvecs[:, 0]  #The ":" means "all rows"

# Output
print("True ground state energy:", ground_energy)
#print("Ground state vector:\n", ground_state)


True ground state energy: -2.23606797749979


In [6]:
print("going to L qubits, scalable SDP relaxation")

going to L qubits, scalable SDP relaxation


In [7]:
#going to L qubits, scalable SDP relaxation

import cvxpy as cp
import numpy as np

def sdp_ground_state_lb(L, h=1, J=.5, use_symmetry=False):
    """
    SDP relaxation for estimating ground state energy LB of a 1D spin chain
    with periodic boundary conditions.
    
    Hamiltonian: H = sum_i h*Z_i + sum_i J*X_i X_{i+1}  (X_{L+1} = X_1 for PBC)
    """
    # Define variables
    Z = [cp.Variable((1, 1), hermitian=True) for _ in range(L)]
    XX = [cp.Variable((1, 1), hermitian=True) for _ in range(L)]         # X_i X_{i+1}
    YX = [cp.Variable((1, 1), hermitian=True) for _ in range(L)]
    XY = [cp.Variable((1, 1), hermitian=True) for _ in range(L)]   

    constraints = []

    # Enforce translational symmetry if specified
    if use_symmetry:
        for i in range(1, L):
            constraints.append(Z[i] == Z[0])
            constraints.append(XX[i] == XX[0])
            constraints.append(YX[i] == YX[0])
            constraints.append(XY[i] == XY[0])

    # Add moment matrix constraints for each neighboring pair (with periodic boundary)
    for i in range(L):
        next_i = (i + 1) % L  # Wrap around for PBC

        # Moment matrix with Z[i], X_i X_{i+1}, Y_i X_{i+1}, deleteting the punultimate row/column in M
        M2 = cp.bmat([
            [1,                 Z[i][0,0],           XX[i][0,0]],
            [Z[i][0,0],         1,                   1j * YX[i][0,0]],
            [XX[i][0,0],        -1j * YX[i][0,0],    1]
        ])
        constraints.append(M2 >> 0)
        #constraints.append(M2 == M2.H)

        # Moment matrix with Z[next_i], X_i X_{i+1}, X_i Z_{i+1}, deleting the second row/column in M
        M3 = cp.bmat([
            [1,                 Z[next_i][0,0],      XX[i][0,0]],
            [Z[next_i][0,0],    1,                   1j * XY[i][0,0]],
            [XX[i][0,0],        -1j * XY[i][0,0],    1]
        ])
        constraints.append(M3 >> 0)
        # constraints.append(M3 == M3.H)

    # Add |value| ≤ 1 constraints for all expectation values
    for var_list in [Z, XX, YX, XY]:
        for var in var_list:
            constraints.append(cp.abs(var) <= 1)

    # Define the energy objective: sum h * Z_i + J * X_i X_{i+1} (with wrap-around)
    h_term = sum(h * Z[i] for i in range(L))
    j_term = sum(J * XX[i] for i in range(L))  # includes (L, 0) term for PBC
    energy_expr = cp.real(h_term + j_term)

    # Solve the SDP
    problem = cp.Problem(cp.Minimize(energy_expr), constraints)
    problem.solve(solver=cp.MOSEK, verbose=False)

    print("Solver status:", problem.status)
    print(f"Ground state energy lower bound with SDP (L={L}, PBC):", problem.value)

    return problem.value

# Example usage
L=8
energy_lb=sdp_ground_state_lb(L=L, h=1, J=.5, use_symmetry=False)  
#J has to be half of the preceding, as the interaction gets counted twice for L=2 with pbc. scheme: o=o.

print("\n L=",L)
print("gnd_energy_lb/L=", energy_lb/L)


Solver status: optimal
Ground state energy lower bound with SDP (L=8, PBC): -8.9442719094949

 L= 8
gnd_energy_lb/L= -1.1180339886868624


In [8]:
#going to L qubits, (non-scalable) exact diagonalization


import numpy as np
from scipy.sparse import kron, identity, csr_matrix
from scipy.sparse.linalg import eigsh  # sparse eigensolver

# Pauli matrices
I = csr_matrix(np.eye(2))
X = csr_matrix(np.array([[0, 1], [1, 0]]))
Y = csr_matrix(np.array([[0, -1j], [1j, 0]]))
Z = csr_matrix(np.array([[1, 0], [0, -1]]))


def build_operator(pauli, site, L):
    """Construct L-qubit operator with `pauli` at position `site`."""
    ops = [I] * L
    ops[site] = pauli
    result = ops[0]
    for op in ops[1:]:
        result = kron(result, op, format='csr')
    return result


def build_two_qubit_operator(pauli1, pauli2, site1, site2, L):
    """Construct L-qubit operator with pauli1 at site1 and pauli2 at site2."""
    ops = [I] * L
    ops[site1] = pauli1
    ops[site2] = pauli2
    result = ops[0]
    for op in ops[1:]:
        result = kron(result, op, format='csr')
    return result

def exact_diagonalization(L, h=1, J=1, pbc=True):
    """Compute ground state energy using exact diagonalization."""
    H = csr_matrix((2**L, 2**L), dtype=np.complex128)

    # On-site terms: h * Z_i
    for i in range(L):
        H += h * build_operator(Z, i, L)

    # Interaction terms: J * X_i X_{i+1}
    for i in range(L):
        j = (i + 1) % L if pbc else i + 1
        if j < L:
            H += J * build_two_qubit_operator(X, X, i, j, L)

    # Compute lowest eigenvalue
    eigval, _ = eigsh(H, k=1, which='SA')  # Smallest Algebraic
    ground_energy = np.real(eigval[0])

    print(f"Ground state energy from exact diagonalization (L={L}, PBC={pbc}): {ground_energy}")
    return ground_energy

# Example usage
L=8
gnd_energy_exact=exact_diagonalization(L=L, h=1, J=.5, pbc=True)

print("L=",L)
print("exact ground state energy/L=", gnd_energy_exact/L)


Ground state energy from exact diagonalization (L=8, PBC=True): -8.509082235140276
L= 8
exact ground state energy/L= -1.0636352793925345


In [9]:
print("Remark: 1D transverse field Ising model can be analytically solved.")
print("Here it serves as a toy model to understand the mechanism of the relaxation, and how it is scalable wrt exact diagonalization")

Remark: 1D transverse field Ising model can be analytically solved.
Here it serves as a toy model to understand the mechanism of the relaxation, and how it is scalable wrt exact diagonalization
